In [ ]:
#Подготовка данных 
import pandas as pd
import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import re

# Загрузка данных
train_data = pd.read_parquet("/content/sample_data/hackaton_train_types_recom.parquet")
test_data = pd.read_parquet("/content/sample_data/hackaton_test_types_recom.parquet")

# Фильтрация данных (если нужно)
train_data = train_data[train_data["target"] == 1]  # Только корректные примеры
test_data = test_data[test_data["target"] == 1]

# Очистка текста
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
    return text

train_data["clean_name"] = train_data["name"].apply(preprocess_text)
test_data["clean_name"] = test_data["name"].apply(preprocess_text)

# Кодирование категорий
le = LabelEncoder()
train_data["category_encoded"] = le.fit_transform(train_data["category_l2"])
test_data["category_encoded"] = le.transform(test_data["category_l2"])

# Разделение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    train_data["clean_name"],
    train_data["category_encoded"],
    test_size=0.2,
    random_state=42
)

In [ ]:
# Создание TF-IDF векторов
tfidf = TfidfVectorizer(
    max_features=5000,  # Можно увеличить для лучшего качества
    stop_words=stopwords.get_stopwords("russian") + stopwords.get_stopwords("english"),
    ngram_range=(1, 2)  # Учитываем униграммы и биграммы
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

# Обучение LightGBM
model = LGBMClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"  # Важно, если классы несбалансированы
)
model.fit(X_train_tfidf, y_train)

# Оценка модели
y_pred = model.predict(X_val_tfidf)
#print(classification_report(y_val, y_pred, target_names=le.classes_))

In [ ]:
# Сохранение модели (третий блок)
import joblib
import pickle

# Сохраняем модель LightGBM
joblib.dump(model, "lightgbm_category_classifier.pkl")

# Сохраняем TF-IDF векторизатор
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

# Сохраняем LabelEncoder (для обратного преобразования категорий)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

In [ ]:
# Использование модели для предсказаний
def predict_category(text, top_n=3):
    # Очистка текста
    clean_text = preprocess_text(text)
    
    # Векторизация текста
    text_tfidf = tfidf.transform([clean_text])
    
    # Предсказание вероятностей
    probas = model.predict_proba(text_tfidf)[0]
    
    # Топ-N категорий
    top_categories_idx = probas.argsort()[-top_n:][::-1]
    top_categories = le.inverse_transform(top_categories_idx)
    top_probas = probas[top_categories_idx]
    
    return list(zip(top_categories, top_probas))

# Пример использования
new_text = "беспроводные наушники с шумоподавлением"
print(predict_category(new_text))